In [ ]:
from google.colab import files
uploaded = files.upload()

Saving archive.zip to archive.zip


In [ ]:
import zipfile
import os

# Ekstrak zip ke direktori dataset
zip_path = '/content/archive.zip'
extract_path = '/content/dataset'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Cek hasil ekstraksi
os.listdir('/content/dataset')


['Images']

In [ ]:
import shutil
from sklearn.model_selection import train_test_split
import glob
import os

# Define base directory for extracted content
dataset_base = '/content/dataset/Images'

# Create new folders for train and validation sets
os.makedirs(os.path.join(dataset_base, 'train', 'glasses'), exist_ok=True)
os.makedirs(os.path.join(dataset_base, 'train', 'no_glasses'), exist_ok=True)
os.makedirs(os.path.join(dataset_base, 'validation', 'glasses'), exist_ok=True)
os.makedirs(os.path.join(dataset_base, 'validation', 'no_glasses'), exist_ok=True)

# Function to split data
def split_data(source_dir, train_dir, val_dir, split_ratio=0.6):
    # Check if source directory exists and contains files
    if not os.path.exists(source_dir):
        print(f"Source directory not found: {source_dir}")
        return
    files = glob.glob(os.path.join(source_dir, '*'))
    if not files:
        print(f"No files found in source directory: {source_dir}")
        return

    train_files, val_files = train_test_split(files, train_size=split_ratio, random_state=42)
    for f in train_files:
        shutil.copy(f, train_dir)
    for f in val_files:
        shutil.copy(f, val_dir)

# Define source directories within the extracted Images folder
glasses_source = os.path.join(dataset_base, 'glasses')
no_glasses_source = os.path.join(dataset_base, 'no_glasses')

# Split data for glasses and no_glasses, using the correct paths
split_data(glasses_source,
           os.path.join(dataset_base, 'train', 'glasses'),
           os.path.join(dataset_base, 'validation', 'glasses'))

split_data(no_glasses_source,
           os.path.join(dataset_base, 'train', 'no_glasses'),
           os.path.join(dataset_base, 'validation', 'no_glasses'))

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = '/content/dataset/Images/train'
val_dir = '/content/dataset/Images/validation'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)


Found 2951 images belonging to 2 classes.
Found 1969 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5925 - loss: 0.6917

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


93/93 ━━━━━━━━━━━━━━━━━━━━ 191s 2s/step - accuracy: 0.5943 - loss: 0.6890 - val_accuracy: 0.9970 - val_loss: 0.0189
Epoch 2/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 176s 2s/step - accuracy: 0.9817 - loss: 0.0648 - val_accuracy: 0.9970 - val_loss: 0.0089
Epoch 3/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 183s 2s/step - accuracy: 0.9884 - loss: 0.0382 - val_accuracy: 0.8547 - val_loss: 0.6841
Epoch 4/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 197s 2s/step - accuracy: 0.9185 - loss: 0.2303 - val_accuracy: 0.9975 - val_loss: 0.0078
Epoch 5/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 187s 2s/step - accuracy: 0.9940 - loss: 0.0245 - val_accuracy: 0.9985 - val_loss: 0.0050
Epoch 6/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 192s 2s/step - accuracy: 0.9929 - loss: 0.0284 - val_accuracy: 0.9934 - val_loss: 0.0344
Epoch 7/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 178s 2s/step - accuracy: 0.9922 - loss: 0.0279 - val_accuracy: 0.9980 - val_loss: 0.0087
Epoch 8/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 177s 2s/step - accuracy: 0.9924 - loss: 0.0201 - val_accuracy: 0.9964 - val_loss: 0.

In [ ]:
model.save('glasses_classifier.h5')


In [ ]:
train_generator.class_indices


{'glasses': 0, 'no_glasses': 1}

In [ ]:
import os
import shutil

# Lokasi folder
g_train = '/content/dataset/Images/train/glasses'
ng_train = '/content/dataset/Images/train/no_glasses'
g_val = '/content/dataset/Images/validation/glasses'
ng_val = '/content/dataset/Images/validation/no_glasses'

# Buat folder temp
os.makedirs('/content/temp_g', exist_ok=True)
os.makedirs('/content/temp_ng', exist_ok=True)

# Tukar isi train
shutil.move(g_train, '/content/temp_g')
shutil.move(ng_train, '/content/temp_ng')
shutil.move('/content/temp_g/glasses', ng_train)
shutil.move('/content/temp_ng/no_glasses', g_train)

# Tukar isi validation
shutil.move(g_val, '/content/temp_g_val')
shutil.move(ng_val, '/content/temp_ng_val')
shutil.move('/content/temp_g_val/glasses', ng_val)
shutil.move('/content/temp_ng_val/no_glasses', g_val)


'/content/dataset/Images/validation/glasses'

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = '/content/dataset/Images/train'
val_dir = '/content/dataset/Images/validation'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)


Found 2951 images belonging to 2 classes.
Found 1969 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6376 - loss: 0.6404

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


93/93 ━━━━━━━━━━━━━━━━━━━━ 181s 2s/step - accuracy: 0.6392 - loss: 0.6378 - val_accuracy: 0.9944 - val_loss: 0.0282
Epoch 2/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 176s 2s/step - accuracy: 0.9757 - loss: 0.0768 - val_accuracy: 0.9954 - val_loss: 0.0171
Epoch 3/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 186s 2s/step - accuracy: 0.9896 - loss: 0.0408 - val_accuracy: 0.9985 - val_loss: 0.0105
Epoch 4/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 178s 2s/step - accuracy: 0.9909 - loss: 0.0384 - val_accuracy: 0.9985 - val_loss: 0.0072
Epoch 5/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 177s 2s/step - accuracy: 0.9931 - loss: 0.0276 - val_accuracy: 0.9980 - val_loss: 0.0070
Epoch 6/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 176s 2s/step - accuracy: 0.9920 - loss: 0.0303 - val_accuracy: 0.9980 - val_loss: 0.0093
Epoch 7/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 201s 2s/step - accuracy: 0.9957 - loss: 0.0171 - val_accuracy: 0.9985 - val_loss: 0.0058
Epoch 8/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 173s 2s/step - accuracy: 0.9471 - loss: 0.1568 - val_accuracy: 0.9970 - val_loss: 0.

In [ ]:
model.save('glasses_classifier_fixed.h5')
